In [1]:
import os
import glob
import sys
import numpy as np

from sklearn.model_selection import train_test_split
import keras
import librosa

import librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

GENRE_DIR = "../data/songData/genres/"
GENRE_LIST = []
GENRE_LIST.append("blues")
GENRE_LIST.append("classical")
GENRE_LIST.append("country")
GENRE_LIST.append("disco")
GENRE_LIST.append("hiphop")
GENRE_LIST.append("jazz")
GENRE_LIST.append("metal")
GENRE_LIST.append("pop")
GENRE_LIST.append("reggae")
GENRE_LIST.append("rock")

Using TensorFlow backend.


In [2]:
def create_data_for_conv(genre_list=GENRE_LIST):
    os.chdir(GENRE_DIR)
    for genre in genre_list:
        glob_wav = os.path.join(genre, "*.wav")
        for fn in glob.glob(glob_wav):
            create_mfcc_3d(fn)

In [3]:
def create_mfcc_3d(fn):
    y, sr = librosa.load(fn)
    y_harmonic, y_pertcussive = librosa.effects.hpss(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=30)
    harmonic_mfcc = librosa.feature.mfcc(y=y_harmonic, sr=sr,n_mfcc=30)
    percus_mfcc = librosa.feature.mfcc(y=y_pertcussive, sr=sr,n_mfcc=30)
    data = []
    data.append(mfcc)
    data.append(percus_mfcc)
    data.append(harmonic_mfcc)
    data = np.array(data)
    
    base_fn, ext = os.path.splitext(fn)
    data_fn = base_fn + ".mfcc3d"
    np.save(data_fn, data)
    print("Written", data_fn)


In [20]:
X = []

fn = "../data/songData/genres/blues/blues.00000.mfcc3d.npy"
mfcc3d = np.load(fn)
X.append(mfcc3d)

fn = "../data/songData/genres/blues/blues.00000.mfcc3d.npy"
mfcc3d = np.load(fn)
mfcc3d = mfcc3d[:,:,:1290]
mfcc3d.shape = 3, 30, 1293
print(mfcc3d)
X.append(mfcc3d)

all_x_data = np.array(X)
print(all_x_data.shape)

ValueError: total size of new array must be unchanged

In [21]:
def create_ceps3d_all_data():
    genre_list = GENRE_LIST
    base_dir = GENRE_DIR
    X = []
    y = []
    for label, genre in enumerate(genre_list):
        for fn in glob.glob(os.path.join(base_dir, genre, "*.mfcc3d.npy")):
            mfcc3d = np.load(fn)
            if mfcc3d[0][0].shape[0] != 1293:
                print("file shape is", mfcc3d.shape)
                print("fn is", fn)
            X.append(mfcc3d)
            y.append(label)

    print("loaded all data")
    all_x_data = np.array(X)
    all_y_data = np.array(y)

    x_data_path = GENRE_DIR + 'x_3d_all_data'
    y_data_path = GENRE_DIR + 'y_3d_all_data'

    np.save(x_data_path, all_x_data)
    np.save(y_data_path, all_y_data)
    print("Written", x_data_path)
    print("Written", y_data_path)

In [10]:
def prepare_mfcc3d_data():
    genre_list = GENRE_LIST
    base_dir = GENRE_DIR
    X = []
    y = []
    for label, genre in enumerate(genre_list):
        for fn in glob.glob(os.path.join(base_dir, genre, "*.mfcc3d.npy")):
            mfcc3d = np.load(fn)
            if mfcc3d[0][0].shape[0] < 1293:
                mfcc3d_formated_data = mfcc3d[:,:,:1293]
                np.save(fn, mfcc3d_formated_data)
                print("recreate data of", fn)

In [6]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'create_dataset_mfcc_3d.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', 'create_dataset_mfcc_3d.ipynb'], returncode=0)

In [22]:
create_ceps3d_all_data()

file shape is (3, 30, 1292)
fn is ../data/songData/genres/classical/classical.00049.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/classical/classical.00051.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/country/country.00003.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/country/country.00004.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/country/country.00007.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/disco/disco.00014.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/hiphop/hiphop.00031.mfcc3d.npy
file shape is (3, 30, 1290)
fn is ../data/songData/genres/hiphop/hiphop.00032.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/rock/rock.00027.mfcc3d.npy
file shape is (3, 30, 1292)
fn is ../data/songData/genres/rock/rock.00038.mfcc3d.npy
loaded all data


ValueError: could not broadcast input array from shape (3,30,1293) into shape (3,30)